### Fuzzy String Matcher (Basic)

In [ ]:
import pandas as pd
import numpy as np
from string_grouper import match_strings, match_most_similar, \
	group_similar_strings, compute_pairwise_similarities, \
	StringGrouper
import cleanco

In [ ]:
## directory info
rootDataPath = 'INSERT YOUR FILEPATH HERE'

In [ ]:
## file read
matchlist = pd.read_csv(f'{rootDataPath}company_matches_pre_cleaning.csv')

In [ ]:
## establish reference list for all matches to be matched against
ref_list = matchlist[['companyId','companyName']].drop_duplicates().copy()

## remove the following words from the company names
pattern = '|'.join(['-','_','technologies','technology','pharmaceuticals','partners', 'international',
'financial services','management','consulting','industries','entertainment', 'enterprises','com'
'networks','life sciences','capital markets','communications','resources','healthcare','therapeutics','health care'])

## replace those words
ref_list['dehyphenName'] = ref_list.companyName.str.replace(pattern,' ')

## apply company name cleaning
ref_list['cleanMain'] = ref_list.apply(lambda x: cleanco.basename(x.dehyphenName), axis=1)

In [ ]:
## establish list of companies to be matched (the reference list is where we'll be matching them)
to_match_list = matchlist[['companyName2']].drop_duplicates().copy().astype(str)
to_match_list['dehyphenName'] = to_match_list.companyName2.str.replace(pattern, ' ')
to_match_list['cleanToMatch'] = to_match_list.apply(lambda x: cleanco.basename(x.dehyphenName), axis=1)

In [ ]:
## break apart the match list to speed things up into blocks of 600k
matchlists_chunks = [to_match_list.cleanToMatch[0:600000].copy(), to_match_list.cleanToMatch[600000:1200000].copy(), to_match_list.cleanToMatch[1200000:].copy()]
matchlists_chunks = [item.astype(str) for item in matchlists_chunks]

In [ ]:
## test to check similarity threshold. going with 0.67 for now.
matches1 = match_strings(ref_list.cleanMain, matchlists_chunks[0], n_blocks='auto', min_similarity=0.67)
matches1.sort_values(by='similarity', ascending=True).head(20)

In [ ]:
## concatenate all of the matches
matches = pd.concat([match_strings(ref_list.cleanMain, item, n_blocks='auto', min_similarity=0.67) for item in matchlists_chunks], axis=0, ignore_index=True)
matches.sort_values(by='similarity', ascending=False)

In [31]:
from sentence_transformers import SentenceTransformer
import faiss
import requests
import json
import pandas as pd
import numpy as np

companyNames = pd.DataFrame({"Company":["Wells Fargo","Sculptor"]})
class Similarity:
    def __init__(self,companyNames):
        self.companyNames = companyNames
        self.TrainFaiss()
        self.getCompanyMatch()
    
    def retrieveData(self):
        url = "https://www.sec.gov/files/company_tickers.json"
        response = requests.get(url)
        json_data = response.json()
        file_name = "company_tickers.json"
        with open(file_name, "w") as json_file:
            json.dump(json_data, json_file)
        print(f"JSON data saved to {file_name}")
        
    def loadData(self):
        file_name = "company_tickers.json"
        with open(file_name, "r") as json_file:
            loaded_json_data = json.load(json_file)
        df = pd.DataFrame(loaded_json_data)
        df = df.transpose()
        df.title = df.title.str.title()
        df = df.rename({"title":"Companies"},axis=1)
        return df
    
    def TrainFaiss(self):
#         df = pd.read_csv("SEC-Data.csv",index_col=0)
        df = self.loadData()
        self.possibleCompanies = df.Companies
        print(self.possibleCompanies)
        self.encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
        vectors = self.encoder.encode(self.possibleCompanies)
        vector_dimension = vectors.shape[1]
        self.index = faiss.IndexFlatL2(vector_dimension)
        faiss.normalize_L2(vectors)
        self.index.add(vectors)
        
    def getCompanyMatch(self):
        matches = []
        for company in self.companyNames["Company"]:
            match = self.predict(company)
            matches.append(match)
        self.companyNames["Matches"] = matches
            
    def predict(self,search_text):
        search_vector = self.encoder.encode(search_text)
        _vector = np.array([search_vector])
        faiss.normalize_L2(_vector)
        k = self.index.ntotal
        distances, ann = self.index.search(_vector, k=k)
        return self.possibleCompanies[ann[0][0]]
    
    

In [32]:

obj = Similarity(companyNames)


0                       Apple Inc.
1                   Microsoft Corp
2                    Alphabet Inc.
3                   Amazon Com Inc
4                      Nvidia Corp
                   ...            
10893                   Lavoro Ltd
10894         Livewire Group, Inc.
10895    Nocturne Acquisition Corp
10896    Nocturne Acquisition Corp
10897          Wm Technology, Inc.
Name: Companies, Length: 10898, dtype: object


In [33]:
obj.companyNames

,Company,Matches
0,Wells Fargo,Wells Fargo & Company/Mn
1,Sculptor,"Sculptor Capital Management, Inc."
